# Creating a voice assistant for knowledge base!

Using Whisper, OpenAI, Eleven Labs, and ActiveLoop.

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ELEVEN_API_KEY = os.getenv("ELEVEN_API_KEY")
ACTIVELOOP_TOKEN = os.getenv("ACTIVELOOP_TOKEN")

## Knowledge Base

### Web scraping content ( Python library articles ) from Hugging Face Hub

In [2]:
# Importing the necessary libraries
import os
import requests
from bs4 import BeautifulSoup
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import re
import warnings
warnings.filterwarnings("ignore")

In [3]:
my_activeloop_org_id = "ACTIVELOOP_TOKEN"
dataset_name = "voice-assistant"
dataset_path= f'hub://{my_activeloop_org_id}/{dataset_name}'

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
def get_documentation_urls():

    return[
        
    ]